In [2]:
%matplotlib inline

In [55]:
path = "/home/ubuntu/nbs/data/dogscats.3/"
model = "lesson1-diy-weights.h5"


In [6]:
from __future__ import division,print_function

import os, json
from glob import glob
import numpy as np
np.set_printoptions(precision=4, linewidth=100)
from matplotlib import pyplot as plt
import utils; reload(utils)
from utils import plots
import vgg16; reload(vgg16)
from vgg16 import Vgg16

In [109]:
if os.path.isfile(model):
     batch_size=4
     vgg = Vgg16()
     vgg.model.load_weights(model)
     vgg.classes = ["cat", "dog"]   
else:
     batch_size=64
     vgg = Vgg16()
     batches = vgg.get_batches(path+'train', batch_size=batch_size)
     val_batches = vgg.get_batches(path+'valid', batch_size=batch_size*2)
     vgg.finetune(batches)
     vgg.fit(batches, val_batches, nb_epoch=1)
     vgg.model.save_weights(model)


In [110]:
pred_batches,pred_gen = vgg.test(path+'test', batch_size=batch_size)

Found 12500 images belonging to 1 classes.


In [164]:
idxs = np.argmax(pred_gen, axis=1)
preds = [ abs(pred_gen[i, idxs[i]] - int(not bool(idxs[i]))) for i in range(len(idxs))]

In [139]:
import re
ids = map(lambda fn: re.sub(r"\D","", fn),pred_batches.filenames)

In [168]:
predictions = zip(ids,preds)

In [170]:
import csv
with open(path + 'submission.csv','wb') as out:
    csv_out=csv.writer(out)
    csv_out.writerow(['id','label'])
    for row in predictions:
        csv_out.writerow(row)